In [ ]:
from gitlab_to_github_migrator import github_utils as ghu
from gitlab_to_github_migrator import gitlab_utils as glu
from gitlab_to_github_migrator import migrator, utils

In [ ]:
ghc = ghu.get_client(utils.get_pass_value("github/api/token"))

In [ ]:
ghc.get_repo("janfr/gitlab-to-github-migrator")

In [ ]:
ghc.get_organization("janfr")

In [ ]:
ghc.get_user().login

In [ ]:
ghc.get_user("janfrederik").login

In [ ]:
ghc.get_user("janfr").login

In [ ]:
ghc.get_user().create_repo("test-repo", private=True, has_issues=True)

In [ ]:
ghc.get_repo("janfrederik/test-repo")

In [ ]:
ghc.get_repo("janfrederik/test-repo").delete()

In [ ]:
m = migrator.GitlabToGithubMigrator("source/project", "target/repo",
                                    gitlab_token=utils.get_pass_value("gitlab/api/ro_token"),
                                    github_token=utils.get_pass_value("github/api/token"))

In [ ]:
m.gitlab_project

In [ ]:
milestones = m.gitlab_project.milestones.list(get_all=True, state="all")

In [ ]:
milestones[0]

In [ ]:
milestones[0].id

In [ ]:
milestones[0].due_date

In [ ]:
l = ghc.get_repo("owner/repo").get_labels()

In [ ]:
l

In [ ]:
len(tuple(l))

In [ ]:
m.gitlab_project.ssh_url_to_repo

In [ ]:
from pathlib import Path

p = Path("/home/janfr/abu/gitlab-to-github-migrator/src/gitlab_to_github_migrator/migrator.py")

In [ ]:
isinstance(p, str)

In [ ]:
import os
import random
import re
import string
import subprocess

In [ ]:
def get_gitlab_token() -> str | None:
    """Get GitLab token from environment or pass."""
    gitlab_token = os.environ.get("SOURCE_GITLAB_TOKEN")
    if not gitlab_token:
        try:
            result = subprocess.run(["pass", "gitlab/api/ro_token"], capture_output=True, text=True, check=True)
            gitlab_token = result.stdout.strip()
        except (subprocess.CalledProcessError, FileNotFoundError):
            pass  # Fall back to anonymous access
    return gitlab_token

In [ ]:
get_gitlab_token()

In [ ]:
gl = glu.get_client(token=get_gitlab_token())
gitlab_project_path = "flaks/flaks-k8s-cluster-bare"
project = gl.projects.get(gitlab_project_path)

In [ ]:
i = project.issues.get(29)

In [ ]:
i.__dict__

In [ ]:
query = """
query GetWorkItemWithChildren($fullPath: ID!, $iid: String!) {
    namespace(fullPath: $fullPath) {
        workItem(iid: $iid) {
            id
            iid
            title
            workItemType {
                name
            }
            widgets {
                type
                ... on WorkItemWidgetHierarchy {
                    children {
                        nodes {
                            id
                            iid
                            title
                            state
                            workItemType {
                                name
                            }
                            webUrl
                        }
                    }
                }
            }
        }
    }
}
"""

variables = {"fullPath": gitlab_project_path, "iid": str(i.iid)}

data = glu.get_graphql_client(token=get_gitlab_token()).execute(query, variable_values=variables)
data

In [ ]:
widgets = data.get("namespace").get("workItem").get("widgets", [])
widgets

In [ ]:
for widget in widgets:
    if widget.get("type") == "HIERARCHY":
        print(widget)
        break

In [ ]:
widget

In [ ]:
widget.get("children", {}).get("nodes", [])